
IMPORTING THE NECESSARY LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

IMPORTING THE DATA AND STORING IT 

In [2]:
df=pd.read_csv(r"F:\STORI_RECCOMENDATION_SYSTEM\STORI_RECC_DATA.csv",encoding= 'unicode_escape')
df.head(20)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


adding a rating column with values as 1 as values for user interaction matrix

In [3]:
df['rating']=1
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,rating
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,1
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1


found that stockcode is unique for a product but color may change

In [4]:
print(df[['Description','StockCode']][df.StockCode=='85123A'])

                               Description StockCode
0       WHITE HANGING HEART T-LIGHT HOLDER    85123A
49      WHITE HANGING HEART T-LIGHT HOLDER    85123A
66      WHITE HANGING HEART T-LIGHT HOLDER    85123A
220     WHITE HANGING HEART T-LIGHT HOLDER    85123A
262     WHITE HANGING HEART T-LIGHT HOLDER    85123A
...                                    ...       ...
539342  CREAM HANGING HEART T-LIGHT HOLDER    85123A
539979  WHITE HANGING HEART T-LIGHT HOLDER    85123A
540217  WHITE HANGING HEART T-LIGHT HOLDER    85123A
540837  CREAM HANGING HEART T-LIGHT HOLDER    85123A
541639  CREAM HANGING HEART T-LIGHT HOLDER    85123A

[2313 rows x 2 columns]


DATA PREPROCESSING

In [5]:
df.shape

(541909, 9)

In [6]:
df.isnull().sum()


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
rating              0
dtype: int64

dropping all the null values

In [7]:
df=df.dropna()
df.shape

(406829, 9)

In [8]:
df.groupby(['StockCode'])['Country'].count()


StockCode
10002       49
10080       21
10120       30
10123C       3
10124A       5
          ... 
D           77
DOT         16
M          465
PADS         4
POST      1196
Name: Country, Length: 3684, dtype: int64

just a code to understand how many product a customer purchased

In [9]:
df.groupby(['CustomerID'])['Country'].count()

CustomerID
12346.0      2
12347.0    182
12348.0     31
12349.0     73
12350.0     17
          ... 
18280.0     10
18281.0      7
18282.0     13
18283.0    756
18287.0     70
Name: Country, Length: 4372, dtype: int64

CREATING USER INTERACTION MATRIX 


In [10]:
user_item_matrix=df.pivot_table(index='CustomerID',columns='StockCode',values='rating')
user_item_matrix=user_item_matrix.fillna(0)
print(user_item_matrix)
user_item_matrix_1=csr_matrix(user_item_matrix.values)#converting into a sparse matrix
#we can create after sorting by country as well

StockCode   10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
CustomerID                                                                     
12346.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
12347.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
12348.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
12349.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
12350.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
...           ...    ...    ...     ...     ...     ...    ...    ...    ...   
18280.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
18281.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
18282.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
18283.0       0.0    0.0    0.0     0.0     0.0     0.0    0.0    0.0    0.0   
18287.0       0.0    0.0    0.0     0.0 

the new csr matrix

In [11]:
print(user_item_matrix_1)

  (0, 2001)	1.0
  (1, 25)	1.0
  (1, 87)	1.0
  (1, 130)	1.0
  (1, 167)	1.0
  (1, 206)	1.0
  (1, 208)	1.0
  (1, 275)	1.0
  (1, 316)	1.0
  (1, 320)	1.0
  (1, 333)	1.0
  (1, 382)	1.0
  (1, 396)	1.0
  (1, 461)	1.0
  (1, 648)	1.0
  (1, 682)	1.0
  (1, 756)	1.0
  (1, 792)	1.0
  (1, 823)	1.0
  (1, 912)	1.0
  (1, 913)	1.0
  (1, 1032)	1.0
  (1, 1035)	1.0
  (1, 1090)	1.0
  (1, 1091)	1.0
  :	:
  (4371, 1916)	1.0
  (4371, 1917)	1.0
  (4371, 2057)	1.0
  (4371, 2090)	1.0
  (4371, 2093)	1.0
  (4371, 2095)	1.0
  (4371, 2098)	1.0
  (4371, 2100)	1.0
  (4371, 2132)	1.0
  (4371, 2194)	1.0
  (4371, 2196)	1.0
  (4371, 2257)	1.0
  (4371, 2486)	1.0
  (4371, 2584)	1.0
  (4371, 2585)	1.0
  (4371, 2677)	1.0
  (4371, 2678)	1.0
  (4371, 2679)	1.0
  (4371, 2871)	1.0
  (4371, 2915)	1.0
  (4371, 3087)	1.0
  (4371, 3190)	1.0
  (4371, 3191)	1.0
  (4371, 3193)	1.0
  (4371, 3298)	1.0


FINDING NEAREST NEIGHBOOURS

In [12]:
knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20, n_jobs=-1)
knn.fit(user_item_matrix_1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

function to return the index of a customerid 


In [13]:
def func_return_index(customer_id):
    for i in range(0,user_item_matrix.shape[0]):
        if(user_item_matrix.index[i]==customer_id):
            return(i)
    print("no index found",customer_id)

   
        
        
        

    
       


input the customer id for which reccomendations are to be made

In [14]:
CID=float(input("enter the customerid"))
query_index=func_return_index(CID)
print(query_index)
neighbors_nearest=10
distances, indices = knn.kneighbors(user_item_matrix_1[query_index, :].reshape(1, -1), n_neighbors =neighbors_nearest)#..
print(distances,indices)



enter the customerid18281.0
4368
[[3.33066907e-16 7.54744264e-01 7.60954278e-01 7.81782110e-01
  7.86191006e-01 7.88846058e-01 7.96346730e-01 7.97969491e-01
  8.04819985e-01 8.04819985e-01]] [[4368  325 1763 2378 2746  716   73 2255  895 1315]]


MAKING RECCOMENDATIONS

In [15]:
user_item_matrix_new=user_item_matrix
for i in range(user_item_matrix_new.shape[1]):
    if(user_item_matrix_new.iloc[query_index,i]==0):
        sum_list=[user_item_matrix_new.iloc[j,i] for j in indices[0]]#??
        average=0.0
        for k in sum_list:
            average=average+k
        average=(average/neighbors_nearest)
        
        user_item_matrix_new.iloc[query_index,i]=average
    elif(user_item_matrix_new.iloc[query_index,i]==1):
        user_item_matrix_new.iloc[query_index,i]=-1
#for i in range(3684):
    #if(user_item_matrix_new.iloc[query_index,i]!=0):
        #print(user_item_matrix_new.iloc[query_index,i])

items_CID=[user_item_matrix_new.iloc[query_index,i] for i in range(user_item_matrix_new.shape[1])]
items_index=sorted(range(len(items_CID)),key= lambda x: items_CID[x],reverse=True )[0:5]
print(items_index)

    


    

[980, 1292, 1487, 1853, 1854]


RETURNING THE TOP FIVE RECCOMENDATIONS

In [16]:
reccom=[user_item_matrix_new.columns[i] for i in items_index]
print("The stockcodes for the top 5 reccomendation for CustomerID",CID,"is",reccom)

The stockcodes for the top 5 reccomendation for CustomerID 18281.0 is ['22077', '22423', '22630', '23009', '23010']
